# Proyek Klasifikasi Penumpang Titanic (KNN)

Notebook ini disusun untuk memenuhi tugas akhir mata kuliah Data Mining.
Tujuan proyek ini adalah memprediksi keselamatan penumpang kapal Titanic berdasarkan fitur seperti umur, kelas tiket, dan biaya, menggunakan algoritma **K-Nearest Neighbors (KNN)**.

## 1. Import Library
Langkah pertama adalah memanggil semua pustaka (library) Python yang dibutuhkan:
* **Pandas**: Untuk membaca dan mengolah data tabel.
* **Numpy**: Untuk operasi matematika.
* **Matplotlib & Seaborn**: Untuk membuat grafik visualisasi data.
* **Sklearn**: Library utama untuk algoritma Machine Learning (KNN, splitting data, evaluasi).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## 2. Upload Dataset (Khusus Google Colab)
Jalankan cell di bawah ini untuk mengunggah file `titanic_clean.csv` dari komputermu ke dalam sesi Google Colab ini.

In [ ]:
from google.colab import files

print("=== Silakan Upload File 'titanic_clean.csv' ===")
uploaded = files.upload()

=== Silakan Upload File 'titanic_clean.csv' ===


## 3. Load & Cek Data
Membaca file CSV yang baru saja diupload ke dalam variabel DataFrame (`df`) dan menampilkan 5 baris pertama untuk memastikan data terbaca dengan benar.

In [ ]:
import io

# Membaca file yang diupload (mengambil nama file secara otomatis)
# Jika dijalankan di lokal/VS Code, cukup gunakan: df = pd.read_csv('titanic_clean.csv')
try:
    filename = next(iter(uploaded))
    df = pd.read_csv(io.BytesIO(uploaded[filename]))
    print(f"\nBerhasil membaca file: {filename}")
except NameError:
    # Fallback jika tidak menggunakan fitur upload colab (file sudah ada di folder)
    df = pd.read_csv('titanic_clean.csv')
    print("\nMembaca file lokal: titanic_clean.csv")

print("\n=== 5 Baris Pertama Data Titanic ===")
print(df.head())

print("\n=== Informasi Dataset ===")
print(f"Jumlah Data: {len(df)} baris")
print(f"Jumlah Fitur: {len(df.columns)} kolom")
print(f"\nNama Kolom: {list(df.columns)}")

## 4. Visualisasi Data
Kita akan menggunakan Scatter Plot untuk melihat pola persebaran data.
Grafik ini menghubungkan **Umur (Age)** dan **Harga Tiket (Fare)**, dengan warna titik yang membedakan antara penumpang yang **Selamat (1)** dan **Tidak Selamat (0)**.

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Age', y='Fare', hue='Survived', data=df, palette='seismic', s=100, alpha=0.7)
plt.title('Sebaran Penumpang Titanic (Umur vs Harga Tiket)', fontsize=14, fontweight='bold')
plt.xlabel('Umur', fontsize=12)
plt.ylabel('Harga Tiket', fontsize=12)
plt.legend(title='Status (1=Selamat)', fontsize=10)
plt.grid(True, alpha=0.3)
plt.show()

## 5. Splitting Data (Membagi Data)
Tahap ini memisahkan data menjadi dua bagian:
1.  **Fitur (X)**: Atribut data (Kelas, Gender, Umur, Keluarga, Harga).
2.  **Target (y)**: Label yang ingin diprediksi (Survived).

Selain itu, data juga dibagi menjadi **Data Latih (80%)** untuk melatih model, dan **Data Uji (20%)** untuk mengukur performa model nanti.

In [ ]:
# X = Fitur (Hapus kolom target 'Survived')
X = df.drop('Survived', axis=1)
# y = Target (Hanya kolom 'Survived')
y = df['Survived']

# Bagi data: 80% Training, 20% Testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("=== PEMBAGIAN DATA ===")
print(f"Jumlah Data Latih: {len(X_train)} ({len(X_train)/len(df)*100:.1f}%)")
print(f"Jumlah Data Uji  : {len(X_test)} ({len(X_test)/len(df)*100:.1f}%)")
print(f"\nFitur yang digunakan: {list(X.columns)}")

## 6. Training Model
Kita melatih algoritma **K-Nearest Neighbors (KNN)**. Di sini kita menggunakan parameter `n_neighbors=5`, yang berarti model akan melihat 5 tetangga terdekat untuk menentukan klasifikasi.

In [ ]:
# Inisialisasi model KNN dengan k=5
model = KNeighborsClassifier(n_neighbors=5)

# Latih model menggunakan data training
model.fit(X_train, y_train)
print("✓ Model KNN berhasil dilatih dengan k=5!")
print(f"✓ Model telah mempelajari {len(X_train)} data penumpang")

## 7. Evaluasi Model & Tabel Hasil
Setelah dilatih, kita minta model untuk memprediksi data uji (`X_test`). Hasil prediksinya dibandingkan dengan kunci jawaban asli (`y_test`) untuk menghitung akurasi. Selain itu, kita menampilkan **Tabel Perbandingan** untuk melihat detail prediksi model.

In [ ]:
# 1. Prediksi data testing
y_pred = model.predict(X_test)

# 2. Hitung akurasi
akurasi = accuracy_score(y_test, y_pred)
print(f"\n{'='*50}")
print(f"AKURASI MODEL: {akurasi * 100:.2f}%")
print(f"{'='*50}")

# 3. MEMBUAT TABEL PERBANDINGAN (DIPERBAIKI)
# Reset index untuk menghindari konflik
df_hasil = X_test.reset_index(drop=True).copy()
df_hasil['Status_Asli'] = y_test.reset_index(drop=True)
df_hasil['Prediksi_Model'] = y_pred

# Mapping Label (0=Tidak Selamat, 1=Selamat)
label_map = {0: 'Tidak Selamat', 1: 'Selamat'}
df_hasil['Status_Asli_Label'] = df_hasil['Status_Asli'].map(label_map)
df_hasil['Prediksi_Label'] = df_hasil['Prediksi_Model'].map(label_map)

# Cek Hasil (Benar/Salah)
df_hasil['Hasil'] = np.where(
    df_hasil['Status_Asli'] == df_hasil['Prediksi_Model'],
    '✓ Benar',
    '✗ Salah'
)

# Tampilkan tabel dengan format yang lebih rapi
print("\n=== TABEL PERBANDINGAN PREDIKSI (15 Data Pertama) ===")
cols_display = ['Pclass', 'Sex', 'Age', 'Fare', 'Status_Asli_Label', 'Prediksi_Label', 'Hasil']
df_display = df_hasil[cols_display].head(15)

# Rename kolom untuk tampilan lebih baik
df_display.columns = ['Kelas', 'Gender', 'Umur', 'Harga', 'Status Asli', 'Prediksi', 'Hasil']

print(df_display.to_string(index=True))

# 4. Statistik Hasil
total_benar = (df_hasil['Hasil'] == '✓ Benar').sum()
total_salah = (df_hasil['Hasil'] == '✗ Salah').sum()

print(f"\n{'='*50}")
print(f"RINGKASAN HASIL:")
print(f"  ✓ Prediksi Benar : {total_benar} dari {len(df_hasil)} ({total_benar/len(df_hasil)*100:.1f}%)")
print(f"  ✗ Prediksi Salah : {total_salah} dari {len(df_hasil)} ({total_salah/len(df_hasil)*100:.1f}%)")
print(f"{'='*50}")

# 5. Confusion Matrix
print("\n=== CONFUSION MATRIX ===")
cm = confusion_matrix(y_test, y_pred)
print("\nFormat: [Tidak Selamat, Selamat]")
print(cm)
print(f"\nPenjelasan:")
print(f"  - True Negative (Benar prediksi Tidak Selamat): {cm[0][0]}")
print(f"  - False Positive (Salah prediksi Selamat): {cm[0][1]}")
print(f"  - False Negative (Salah prediksi Tidak Selamat): {cm[1][0]}")
print(f"  - True Positive (Benar prediksi Selamat): {cm[1][1]}")

# 6. Tampilkan laporan klasifikasi detail
print("\n=== LAPORAN KLASIFIKASI DETAIL ===")
print(classification_report(y_test, y_pred, target_names=['Tidak Selamat', 'Selamat']))

## 8. Visualisasi Confusion Matrix
Membuat visualisasi grafis dari Confusion Matrix untuk mempermudah interpretasi hasil prediksi model.

In [ ]:
# Plot Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True,
            xticklabels=['Tidak Selamat', 'Selamat'],
            yticklabels=['Tidak Selamat', 'Selamat'])
plt.title('Confusion Matrix - Prediksi Model KNN', fontsize=14, fontweight='bold')
plt.ylabel('Status Asli', fontsize=12)
plt.xlabel('Prediksi Model', fontsize=12)
plt.show()

## 9. Simulasi Prediksi Manual
Di sini kita mencoba menguji model dengan data buatan sendiri (data Jack dan Rose) untuk melihat bagaimana model memprediksi kasus spesifik berdasarkan pola yang sudah dipelajari.

In [ ]:
print("\n" + "="*60)
print("SIMULASI PREDIKSI: Jack vs Rose")
print("="*60)

# Format Input Sesuai Kolom: [Pclass, Sex, Age, SibSp, Parch, Fare]
# Keterangan: Sex 0 = Pria, 1 = Wanita

# Data Jack: Kelas 3 (Ekonomi), Pria, Umur 20, Sendiri (0 sib, 0 par), Tiket Murah
data_jack = [3, 0, 20, 0, 0, 7.5]

# Data Rose: Kelas 1 (Eksekutif), Wanita, Umur 17, Bawa Ortu (1 par), Tiket Mahal
data_rose = [1, 1, 17, 0, 1, 100.0]

# Buat DataFrame baru untuk inputan ini
input_baru = pd.DataFrame([data_jack, data_rose], columns=X.columns)

# Lakukan prediksi
hasil_prediksi = model.predict(input_baru)
probabilitas = model.predict_proba(input_baru)

# Tampilkan hasil dengan detail
print("\nDetail Input Data:")
print("-" * 60)

penumpang = [
    {"nama": "Jack", "data": data_jack},
    {"nama": "Rose", "data": data_rose}
]

for i, p in enumerate(penumpang):
    status = "SELAMAT ✓" if hasil_prediksi[i] == 1 else "TIDAK SELAMAT ✗"
    prob_tidak_selamat = probabilitas[i][0] * 100
    prob_selamat = probabilitas[i][1] * 100

    print(f"\n{p['nama'].upper()}:")
    print(f"  - Kelas Tiket : {p['data'][0]} ({'Eksekutif' if p['data'][0]==1 else 'Ekonomi'})")
    print(f"  - Gender      : {'Wanita' if p['data'][1]==1 else 'Pria'}")
    print(f"  - Umur        : {p['data'][2]} tahun")
    print(f"  - Keluarga    : {p['data'][3]} saudara, {p['data'][4]} orangtua")
    print(f"  - Harga Tiket : ${p['data'][5]}")
    print(f"  → PREDIKSI    : {status}")
    print(f"  → Probabilitas: Tidak Selamat {prob_tidak_selamat:.1f}% | Selamat {prob_selamat:.1f}%")

print("\n" + "="*60)

## 10. Ekspor Hasil Prediksi (Opsional)
Menyimpan hasil prediksi lengkap ke dalam file CSV untuk analisis lebih lanjut.

In [ ]:
# Simpan hasil prediksi ke CSV
df_hasil_export = df_hasil[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
                             'Status_Asli_Label', 'Prediksi_Label', 'Hasil']].copy()
df_hasil_export.columns = ['Kelas', 'Gender', 'Umur', 'Saudara', 'Orangtua',
                           'Harga', 'Status_Asli', 'Prediksi', 'Hasil_Cek']

# Simpan ke file
df_hasil_export.to_csv('hasil_prediksi_titanic.csv', index=False)
print("✓ Hasil prediksi berhasil disimpan ke 'hasil_prediksi_titanic.csv'")
print(f"✓ Total data yang disimpan: {len(df_hasil_export)} baris")